In [2]:
import numpy as np
import functions as func
import matplotlib.pyplot as plt
import pdb, glob, mpld3, time
import mirpyidl as idl
from tqdm import tqdm
from astropy.io import fits, ascii
from astropy.wcs import WCS
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Table
from mpld3 import plugins

%matplotlib inline
plt.rcParams["figure.figsize"] = (10, 8)
mpld3.enable_notebook()

**File name directory example:  
bd +60 1753 all ch1 data: /data1/phot_cal/spitzer/bd601753/r*/ch1/bcd/*_bcd.fits**

In [4]:
fnames = np.sort(glob.glob(raw_input('path to bcd files:')))
len(fnames)

path to bcd files:/data1/phot_cal/spitzer/bd601753/r*/ch1/bcd/*_bcd.fits


19742

**Next cell generates flux data with no systematics removed**

In [3]:
sky = SkyCoord('17 24 52.2772360943 +60 25 50.780790994', unit=(u.hourangle, u.deg))

#Issues list
crd_conversion = []
centroiding    = []
bad_cen_guess  = []

data = Table(names = ('File#','ACenX', 'ACenY', 'FCenX', 'FCenY', 'Time[MJD]', 'Raw_Flux', 'Bkg_Flux', 'Res_Flux'), 
             dtype = ('S15', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8'))


for fn in tqdm(fnames):
    
    hdu    = fits.open(fn)
    header = hdu[0].header
    image  = hdu[0].data
    hdu.close()

    fnum = fn[62:75]
    Time = header['MJD_OBS']

    try:
        w = WCS(header)
        pix = sky.to_pixel(w)
    except:
        crd_conversion.append(fnum)
        continue

    if (pix[0]>0) & (pix[0]<256) & (pix[1]>0) & (pix[1]<256):

        try:
            cenX, cenY = func.gen_center_g2d(pix[0], pix[1], 7, 5, 4, 4, 0, image)
        except:
            centroiding.append(fnum)
            continue
        
        if (np.abs(cenX - pix[0]) <= 2) & (np.abs(cenY-pix[1]) <= 2):
            
            try:
                # Extracting raw flux
                raw_flux, src_ap = func.photometry(image, [cenX], [cenY], rad = 10)

                # Extrating a mean background flux
                bkg, bkg_ap = func.photometry(image, [cenX], [cenY], shape = 'CircAnn', r_in = 12, r_out = 20)
                bkg_mean = bkg/bkg_ap.area()
                bkg_flux = bkg_mean*src_ap.area()

                # Subtracting background
                res_flux  = raw_flux - bkg_flux

                data.add_row([fnum, pix[0], pix[1], cenX, cenY, Time, raw_flux, bkg_flux, res_flux])
            
            except:
                continue
            
        else:
            bad_cen_guess.append(fnum)

100%|██████████| 19742/19742 [1:10:01<00:00,  4.70it/s]


**Run next cell if you want to save the data generated from previous cell  
Change file names if you don't want to overwrite**

In [ ]:
np.save('crd_conversion_issue_bd_ch1.npy', np.array(crd_conversion))
np.save('centroiding_issue_bd_ch1.npy', np.array(centroiding))
np.save('bad_cen_guess_bd_ch1.npy', np.array(bad_cen_guess))
ascii.write(data, 'bd_ch1_refined_data.csv')

**Remove systematics using IDL code  
Systematics removed through this process: Array location dependent correction, pixel phase correction, callibration factor**

```idl
data  = read_csv('/bd_ch1_data.csv')    ;Use proper file path
cenX  = data.field4                     ;Usually column 4 is the fitted cenX column of my generated table
cenY  = data.field5                     ;Be careful about which column you are using
oFlux = data.field9                     ;We use the residual flux column for observed flux

;Now to calculate corrected flux
;Make sure to have irac_aphot_corr_cryo.pro in the same folder
;You might need to comple it using .compile irac_aphot_corr_cryo.pro command 
cFlux = IRAC_APHOT_CORR_CRYO(obsFlux, cenX, cenY, 1)
write_csv, filename.csv, cFlux

;Useful commands:
;help, data  (works with any array)
;print, data 
```

**Use last systematics removed flux to remove another systematics: Aperture correction**

In [5]:
corr_flux = ascii.read('../hd_ch1_cFlux.csv')  #Use proper filename

In [6]:
# Find the aperture correction factor from the following link:
# https://irsa.ipac.caltech.edu/data/SPITZER/docs/irac/iracinstrumenthandbook/27/
#from the table at the end of that link, select your value accoring to your aperture size and channel

ap_corr = 1.000

In [9]:
#write the refined flux to the data table in a new column

data = ascii.read('hd_ch1_data.csv')   #Again.. use proper filename
data['Refined_Flux'] = np.array(corr_flux).astype('Float64')*ap_corr
ascii.write(data, 'hd_ch1_refined_data.csv', overwrite = True) #change filename if overwriting is not desired

**Go from MJy/Sr to Jy**